In [1]:
import tensorflow as tf

2023-06-06 21:22:21.447936: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 21:22:24.327439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
# create CNN model for egg recognition for input image size 500x500 from RGB to grayscale but float32 type, then with label healthy or unhealthy and (1, 2, 3, 4, 5, 6) as size of the egg
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(370, 370, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=6, activation='softmax')
])

# compile the model with adam optimizer and loss function categorical crossentropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# print the summary of the model
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 368, 368, 16)      160       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 184, 184, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 182, 182, 32)      4640      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 91, 91, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 89, 89, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 44, 44, 64)      

In [35]:
# get the image datasets from the datasets/grayscale folder where the image name is the label like unhealthy-1.png
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

# get the image datasets from the datasets/grayscale folder where the image name is the label like unhealthy-1.png
y_labels = ['healthy-1', 'healthy-2', 'healthy-3', 'healthy-4', 'healthy-5', 'healthy-6', 'unhealthy-1', 'unhealthy-2', 'unhealthy-3', 'unhealthy-4', 'unhealthy-5', 'unhealthy-6']

# get the image datasets from the datasets/grayscale folder where the image name is the label like unhealthy-1.png
train_generator = train_datagen.flow_from_directory(
        'datasets/grayscale',
        target_size=(370, 370),
        batch_size=32,
        class_mode='categorical',
        color_mode='grayscale')

# train the model with 10 epochs
model.fit(
        train_generator,
        steps_per_epoch=100,
        epochs=100,
        verbose=1)


Found 1 images belonging to 1 classes.
Epoch 1/100


2023-06-06 21:58:35.662790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - 1s 301us/step - loss: 19.3137 - accuracy: 1.0000


In [36]:
# save the model to the model folder
model.save('model/ESGR.h5')

In [37]:
# load the model from the model folder
tmodel = tf.keras.models.load_model('model/ESGR.h5')

# load the image from the datasets/test folder
img = tf.keras.preprocessing.image.load_img('datasets/grayscale/unhealthy-1/unhealthy-1.png', target_size=(370, 370), color_mode='grayscale')

# convert the image to array
img_array = tf.keras.preprocessing.image.img_to_array(img)

# reshape the image array
img_array = tf.expand_dims(img_array, 0)

# predict the image
predictions = tmodel.predict(img_array)

# get the label of the prediction
score = tf.nn.softmax(predictions[0])

import numpy as np

# print the label of the prediction
print(np.argmax(score))


1/1 [==============================] - 0s 119ms/step
0
